In [1]:
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
from PIL import Image, ImageTk
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from ocr_model import model

C:\Users\Amirreza_Bazmi\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# **Some functions about image**

In [2]:
x_size, y_size = 32, 32

def load_image(path):
    img = np.expand_dims(cv2.resize(cv2.imread(path, 0), (x_size, y_size)), axis=2)
    return img

def show_image(caption, img, destroy=True, show=True, wait_ms=1000):
    if not show: return
    cv2.imshow(caption, img)
    cv2.waitKey(wait_ms)
    if destroy: cv2.destroyAllWindows()


# **Creating and Loading the model**

In [3]:
model = model
model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

model = load_model("best_accuracy_model.keras")

# **Preprocessing new image function**

In [4]:
def load_image(path, show_process=True):
    image = cv2.imread(path)
    # Base image
    show_image(caption="Original image", img=image, show=show_process)
    original_img = image.copy()

    # Bitwise, Gray scale, Blur
    min_ffill_count = 50  # Filtering noise
    number_str = ""
    img_base = cv2.bitwise_not(image)
    img_gray = cv2.cvtColor(img_base, cv2.COLOR_BGR2GRAY)
    img_blurred = cv2.blur(img_gray, ksize=(5, 5))
    show_image("Blurred image", img=img_blurred, show=show_process)

    # Binary image
    _, binay_img = cv2.threshold(img_blurred, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    show_image("Binary image", img=binay_img, show=show_process)

    # Rotate the binary image
    rotated_binary_img = cv2.rotate(binay_img, cv2.ROTATE_90_CLOCKWISE)
    show_image(caption="Rotated binary image", img=rotated_binary_img, show=show_process)

    # Height of rotated image
    height = rotated_binary_img.shape[0]

    # Create a copy for processing
    processed_img = rotated_binary_img.copy()

    # Finding the number character
    for x in range(height):
        # Check if there are any white pixels left in the image
        if np.max(processed_img) == 0:
            break  

        # Check entire row for white pixels
        text_pixel = np.where(processed_img[x, :] == 255)[0]
        if text_pixel.size > 0:
            y = (text_pixel[0])
            img_ffill = processed_img.copy()

            # Flood Fill
            ffill_count, ffill_img, ffill_mask, ffill_rect = cv2.floodFill(img_ffill, mask=None, seedPoint=(y, x), newVal=64)
            # ffill_count, ffill_img, ffill_mask, ffill_rect = cv2.floodFill(img_ffill, mask=None, seedPoint=(y, x), newVal=0)
            show_image("Current digit:", img_ffill, destroy=False, show=show_process)

            img_diff = processed_img - img_ffill
            img_diff[img_diff != 0] = 255
            show_image("Current digit:", img_diff, destroy=False, show=show_process)

            # Extract the digit using the bounding rectangle
            x1, y1, w, h = ffill_rect
            img_digit = img_diff[y1:y1+h, x1:x1+w]
            img_digit = cv2.rotate(img_digit, cv2.ROTATE_90_COUNTERCLOCKWISE)

            # Skip if the digit is too small (likely noise)
            if w < 5 or h < 5:
                continue

            img_digit_resized = np.array([cv2.resize(img_digit, (x_size, y_size))]) / 255
            img_digit_resized = np.expand_dims(img_digit_resized, axis=3)

            # Predicting the number of image
            predicted = model.predict(img_digit_resized)
            y_predicted = np.argmax(predicted[0])
            print(f"Predict: {y_predicted}")

            if ffill_count > min_ffill_count:
                number_str += str(y_predicted)

            show_image("Current digit: ", img_digit, destroy=False, show=show_process)

            # Remove the detected digit from the processed image
            processed_img[y1:y1+h, x1:x1+w] = 0

    # Create a white strip at the bottom of the original image
    height_strip = 50
    # Create a white strip with the same width as the image and 3 channels (RGB)
    white_strip = np.ones((height_strip, original_img.shape[1], 3), dtype=np.uint8) * 255

    # Concatenate the original image and the white strip vertically
    result_img = np.vstack([original_img, white_strip])

    # Add text to the result image
    cv2.putText(result_img, "Number:-" + number_str, org=(10, result_img.shape[0] - 15),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 0, 0), lineType=cv2.LINE_AA)

    cv2.destroyAllWindows()
    show_image("Final digit", result_img, show=show_process)
    print(number_str)
    return (number_str, result_img)

# **GUI**

In [5]:
def open_file_dialog():
    # Define the supported file types
    file_types = [("All image files", "*.png;*.jpg;*.bmp"), ("PNG Files", "*.png"), ("JPG Files", "*.jpg"), ("BMP Files", "*.bmp")]

    # Open file dialog
    file_path = tk.filedialog.askopenfilename(filetypes=file_types)

    # Check if a file was selected
    if file_path:
        # Update status label to indicate processing is in progress
        lbl_no.config(text="Working...")
        root.update()  # Force GUI update to show the status immediately

        # Get checkbox value (1 if checked, 0 if unchecked) to determine if processing steps should be shown
        show_process = chk_var.get() == 1

        # Call the image processing function to recognize numbers in the image
        # Returns: (recognized_number_string, result_image_with_text)
        the_no, the_img = load_image(file_path, show_process=show_process)

        # Update the label to show the recognized number
        lbl_no.config(text="The number is: " + the_no)

        # Convert image color
        the_img = cv2.cvtColor(the_img, cv2.COLOR_BGR2RGB)

        # Convert NumPy array to PIL Image object
        img_pil = Image.fromarray(the_img)

        # Convert PIL Image to Tkinter-compatible PhotoImage
        img_tk = ImageTk.PhotoImage(img_pil)

        # Update the image label to display the result image
        lbl_img.config(image=img_tk)
        # Keep a reference to prevent garbage collection
        lbl_img.image = img_tk

In [6]:
# Main application window
root = tk.Tk()
# Set the window title
root.title("Number reader")

# Get the screen dimensions
screen_width = root.winfo_screenwidth()  # Width of the screen
screen_height = root.winfo_screenheight()  # Height of the screen

# Define the desired window dimensions
window_width = 860
window_height = 640

# Calculate the x and y coordinates to center the window
x_coordinate = int((screen_width / 2) - (window_width / 2))  # Center horizontally
y_coordinate = int((screen_height / 2) - (window_height / 2))  # Center vertically

# Set the window geometry
root.geometry(f"{window_width}x{window_height}+{x_coordinate}+{y_coordinate}")

# Create a button to load images
btn_load = tk.Button(root, text="Load image", command=open_file_dialog)
# Pack the button in the window
btn_load.pack(anchor="center", pady=10)

# Create an integer variable to store the checkbox state
chk_var = tk.IntVar()
# Create a checkbox
chk_show = tk.Checkbutton(root, text="Show process", variable=chk_var)
# Pack the checkbox
chk_show.pack()
# Set the checkbox to checked by default (value 1)
chk_var.set(1)

# Create a label to display the recognized number
lbl_no = tk.Label(root, text="")
# Pack the label
lbl_no.pack(anchor="center", pady=10)

# Create a label to display the processed image
lbl_img = tk.Label(root)
# Pack the label
lbl_img.pack(anchor="center", pady=10)

# Start the Tkinter event loop to display
root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
Predict: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Predict: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Predict: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Predict: 3
0123
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predict: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Predict: 8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Predict: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Predict: 4
3824
